In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml import Pipeline

'''
docker start spark

127.0.0.1:8888/?token=c931dec8954620e490c7fa48529b766a83e7dcb47e9ec19a
'''

'\ndocker start spark\n\n127.0.0.1:8888/?token=c931dec8954620e490c7fa48529b766a83e7dcb47e9ec19a\n'

In [3]:
!uname -a

Linux 890d146aa60d 4.15.0-46-generic #49-Ubuntu SMP Wed Feb 6 09:33:07 UTC 2019 x86_64 x86_64 x86_64 GNU/Linux


In [4]:
spark = SparkSession.builder.appName('teste').getOrCreate()
sc = spark.sparkContext

In [3]:
df = spark.read.csv('data/train.csv', header=True, inferSchema=True)
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [4]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

In [5]:
categoricalColumns = ['Sex']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, 
                                  outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], 
                                     outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
    
label_stringIdx = StringIndexer(inputCol = 'deposit', outputCol = 'label')
stages += [label_stringIdx]
numericCols = ['Age']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [6]:
# pipeline = Pipeline(stages = stages)
# pipelineModel = pipeline.fit(df)
# df = pipelineModel.transform(df)
# selectedCols = ['label', 'features'] + cols
# df = df.select(selectedCols)
# df.printSchema()

In [10]:
myRange = spark.range(1000).toDF('number')

In [14]:
divisBy2 = myRange.where('number % 2 = 0')

In [16]:
divisBy2.count()

500

In [32]:
titanic = spark.read.option('interSchema', 'true').option('header', 'true').csv('data/train.csv')

In [33]:
titanic.sort('Sex').explain()

== Physical Plan ==
*(2) Sort [Sex#170 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(Sex#170 ASC NULLS FIRST, 200)
   +- *(1) FileScan csv [PassengerId#166,Survived#167,Pclass#168,Name#169,Sex#170,Age#171,SibSp#172,Parch#173,Ticket#174,Fare#175,Cabin#176,Embarked#177] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/jovyan/git/personal/lab/spark101/data/train.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<PassengerId:string,Survived:string,Pclass:string,Name:string,Sex:string,Age:string,SibSp:s...


In [37]:
#SQL mode
titanic.createOrReplaceTempView('titanic')
sqlWay = spark.sql('Select Cabin from titanic')

In [38]:
sqlWay

DataFrame[Cabin: string]

In [58]:
from pyspark.sql.functions import desc
a  = titanic.groupBy('Sex').count().withColumnRenamed('count("Sex")', 'soma').collect()

In [59]:
flightData2015\
.groupBy(“DEST_COUNTRY_NAME”)\
.sum(“count”)\
.withColumnRenamed(“sum(count)”, “destination_total”)\
.sort(desc(“destination_total”))\
.limit(5)\
.collect()

[Row(Sex='female', count=314), Row(Sex='male', count=577)]

In [63]:
staticDataFrame = spark.read.format('csv').option('header', 'true')\
.option('inferSchema', 'true').load('data/train.csv')

staticDataFrame.createOrReplaceTempView('dead_data')
staticSchema = staticDataFrame.schema

streamingDataFrame = spark.readStream.schema(staticSchema)\
.option('maxFilesPerTrigger', 1).format('csv')\
.option('header', 'true').load('data/train.csv')

In [64]:
streamingDataFrame

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]